<a href="https://colab.research.google.com/github/SamuelaAnastasi/Char_Level_LSTM/blob/master/Udacity_Char_Level_LSTM_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F

##Load data and tokenize

In [0]:
# read text file
with open('/gdrive/My Drive/Colab Notebooks/char_lstm/anna.txt', 'r') as f:
  text = f.read()

In [0]:
print(text[:100])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin


In [0]:
print(text_unique[:6])

('.', 'r', 'c', 'G', '1', 'K')


In [0]:
for k, v in int_to_text.items():
  if k < 6:
    print("Key : {} , Value : {}".format(k,v))


Key : 0 , Value : .
Key : 1 , Value : r
Key : 2 , Value : c
Key : 3 , Value : G
Key : 4 , Value : 1
Key : 5 , Value : K


In [0]:
print({k: text_to_int[k] for k in list(text_to_int)[:6]})

{'.': 0, 'r': 1, 'c': 2, 'G': 3, '1': 4, 'K': 5}


In [0]:
print(encoded_text[:6])

[18 76 22 55 75 63]


###Preprocess data

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    print(one_hot.shape)
    
#     Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    print(one_hot.shape)
#     print(arr.flatten.shape)
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    print(one_hot.shape)
    
    return one_hot

In [0]:
# check that the function works as expected
# Initialize the encoded array with 0
test_seq1 = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq1, 8)

print(one_hot)
# print(test_seq1.shape)
# print(one_hot.shape)

(3, 8)
(3, 8)
(1, 3, 8)
[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
# check that the function works as expected
# Fill the appropriate elements with ones
test_seq2 = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq2, 8)

print(one_hot)
print(test_seq2.shape)
print(one_hot.shape)

(3, 8)
(3, 8)
(1, 3, 8)
[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]
(1, 3)
(1, 3, 8)


In [0]:
# check that the function works as expected
# Finally reshape it to get back to the original array
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)
# print(test_seq1.shape)
print(one_hot.shape)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded_text, 8, 50)
x, y = next(batches)

In [0]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[18 76 22 55 75 63  1 81  4 56]
 [44 13  6 81 75 76 22 75 81 22]
 [63  6 34 81 13  1 81 22 81 57]
 [44 81 75 76 63 81  2 76 66 63]
 [81 44 22 30 81 76 63  1 81 75]
 [ 2  7 44 44 66 13  6 81 22  6]
 [81 26  6  6 22 81 76 22 34 81]
 [59  9 23 13  6 44 28 79  0 81]]

y
 [[76 22 55 75 63  1 81  4 56 56]
 [13  6 81 75 76 22 75 81 22 75]
 [ 6 34 81 13  1 81 22 81 57 13]
 [81 75 76 63 81  2 76 66 63 57]
 [44 22 30 81 76 63  1 81 75 63]
 [ 7 44 44 66 13  6 81 22  6 34]
 [26  6  6 22 81 76 22 34 81 44]
 [ 9 23 13  6 44 28 79  0 81 49]]


In [0]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [0]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [0]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
(12800, 83)
(12800, 83)
(128, 100, 83)
Epoch: 1/20... Step: 10..

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
print(sample(net, 1000, prime='Anna', top_k=5))

(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(

In [0]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(1, 83)
(1, 83)
(1, 1, 83)
(